# 필요 구성 라이브러리 다운로드

In [101]:
# !pip install --upgrade pip
# !pip install konlpy
# !pip install nltk
# !pip install sklearn
# !pip install pandas
# !pip install jellyfish
# !pip install segtok
# !pip install networkx
# !pip install matplotlib
# !pip install tensorflow
# !pip install krwordrank
# !pip install soynlp
# !pip install textrank

  Using cached textrank-0.1.0.tar.gz (2.8 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for textrank: filename=textrank-0.1.0-py3-none-any.whl size=3420 sha256=67a2e691b30f0665a0a1b503d4b4c007d1f730fe97c8d47df9f22e774ed5ffc8
  Stored in directory: /Users/bigtree/Library/Caches/pip/wheels/a6/fb/af/a1b9bfa5de21f67e6fa6edf7d5f3ab2f9e26feb5ce97a4edc8
Successfully built textrank


# 텍스트 마이닝 실험

In [121]:
# 필요 라이브러리 선언
from pprint import pprint

from yake import korea_token
from yake.korea_token import edit_josa, edit_sentences
import math

from konlpy.tag import Komoran
from collections import Counter

import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from math import log
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
np.random.seed(seed=0)

from krwordrank.word import KRWordRank
from krwordrank.sentence import summarize_with_sentences
from krwordrank.word import summarize_with_keywords

from scipy.sparse import csr_matrix
from soynlp.tokenizer import MaxScoreTokenizer
from sklearn.metrics import pairwise_distances

# from textrank import KeywordSummarizer

In [61]:
STRTexT = "러시아가 우크라이나 키이우 등을 공습한 것은 군 내부 비판과 블라디미르 푸틴 러시아 대통령의 자존심 때문이라는 분석이 나왔다. 영국 가디언지는 10일(현지시간) 푸틴 대통령의 대규모 공습은 국내 군 비판세력, 러시아가 침공에서 실패하고 있다는 사실, 크림대교 폭발 후 상처받은 자존심에 대한 절박한 답변이라고 풀이했다. 가디언에 따르면 미국 싱크탱크 카네기국제평화재단의 선임 연구원 안드레이 콜레스니코프는 '지금 푸틴이 하는 것은 사소한 복수'라며 '개인적 복수도 있다'고 말했다. 러시아 전쟁 전문가와 군사 블로거 등은 수개월간 우크라이나를 상대로 전면전을 벌이라고 촉구해왔고 키이우 등의 거리에 시신이 있는 끔찍한 사진이 나오자 지금은 만족해한다. 최근 러시아군 수뇌부를 비판했던 람잔 카디로프 체첸 공화국 수장은 '(볼로디미르) 젤렌스키(우크라이나 대통령), 우리는 러시아가 아직 제대로 시작도 안했다고 경고했다'라며 '이제 전쟁 진행에 100% 만족한다'고 말했다. 푸틴 대통령은 이날 러시아 안전보장이사회 회의를 주재하고 크림대교 폭발의 배후로 우크라이나 정보 당국을 겨냥하면서 '이런 종류의 범죄에 대응을 하지 않는 것은 불가능하다'고 말했다. 그러나 드미트로 쿨레바 우크라이나 외무장관은 '러시아는 크림대교 사건 전에도 우크라이나에 계속 미사일 공격을 했다'며 '푸틴은 전투 패배로 절박한 상황이며 전황을 유리하게 바꾸려고 미사일 공포를 사용한다'고 반박했다. 우크라이나 구조대원들이 10일(현지시간) 수도 키이우 시내에서 러시아군의 미사일 공격을 당한 현장을 조사하고 있다. 이날 오전에 키이우 시내에 여러차례 폭발이 발생했다. 푸틴 대통령은 이번 공격이 국방부의 요청에 따라 이뤄졌다고 주장했는데, 이것이 사실이라면 이는 새로운 합동군 총사령관 세르게이 수로비킨의 첫번째 결정이라고 가디언은 전했다. 수로비킨 사령관과 함께 일했던 전 국방부 관계자는 가디언지에 '오늘 키이우에서 벌어진 일이 놀랍지 않았다. 그는 매우 무자비하고 사람 목숨을 신경 쓰지 않는다'며 '그의 손이 우크라이나인의 피로 뒤덮일까 걱정된다'고 말했다. 그러나 푸틴 대통령이 이날 공습으로 얻은 매파들의 호평은 오래가지 않을 수 있다. 그는 이번과 같은 대규모 미사일 공격은 러시아 영토 공격 시 대응으로 남겨둘 것이라고 말했는데 강경파들은 전면전을 원하고 있다. 정치 평론가인 세르게이 마르코프는 '러시아 여론은 대규모 공격과 우크라이나군이 사용할 가능성이 있는 인프라 완전파괴를 원한다'고 말했다. 한편으론 비판 세력의 목소리는 그다지 의미가 없을 수도 있어 보인다. 전쟁과 관련한 의사결정 과정은 여전히 불투명하다. 콜레스니코프 선임 연구원은 '푸틴 대통령으로선 매파와 극보수파의 불만에 대응하는 것이 중요하겠지만 나는 그들의 영향력을 과장하진 않을 것'이라며 '푸틴 자신이 가장 매파적이고 극보수적인 인물'이라고 말했다. 러시아에서 나오는 한 가지 이론은 푸틴 대통령이 악명 높은 새로운 군사령관을 임명함으로써 전쟁에서 국방부의 성과에 대한 분노를 줄이려고 한다는 것이다. 수로비킨 사령관과 2020년까지 함께 일했던 전 공군 중위인 글렙 이리소프는 '수로비킨은 강경파들을 선호하고 와그너 용병회사와도 좋은 관계를 유지한다'며 '그러나 그가 매우 잔인한 동시에 유능한 사령관이지만 모든 문제를 풀 수는 없을 것'이라고 말했다."
LISTText = ["러시아가 우크라이나 키이우 등을 공습한 것은 군 내부 비판과 블라디미르 푸틴 러시아 대통령의 자존심 때문이라는 분석이 나왔다.",
            "영국 가디언지는 10일(현지시간) 푸틴 대통령의 대규모 공습은 국내 군 비판세력, 러시아가 침공에서 실패하고 있다는 사실, 크림대교 폭발 후 상처받은 자존심에 대한 절박한 답변이라고 풀이했다.",
            "가디언에 따르면 미국 싱크탱크 카네기국제평화재단의 선임 연구원 안드레이 콜레스니코프는 '지금 푸틴이 하는 것은 사소한 복수'라며 '개인적 복수도 있다'고 말했다.",
            "러시아 전쟁 전문가와 군사 블로거 등은 수개월간 우크라이나를 상대로 전면전을 벌이라고 촉구해왔고 키이우 등의 거리에 시신이 있는 끔찍한 사진이 나오자 지금은 만족해한다.",
            "최근 러시아군 수뇌부를 비판했던 람잔 카디로프 체첸 공화국 수장은 '(볼로디미르) 젤렌스키(우크라이나 대통령), 우리는 러시아가 아직 제대로 시작도 안했다고 경고했다'라며 '이제 전쟁 진행에 100% 만족한다'고 말했다.",
            "푸틴 대통령은 이날 러시아 안전보장이사회 회의를 주재하고 크림대교 폭발의 배후로 우크라이나 정보 당국을 겨냥하면서 '이런 종류의 범죄에 대응을 하지 않는 것은 불가능하다'고 말했다.",
            "그러나 드미트로 쿨레바 우크라이나 외무장관은 '러시아는 크림대교 사건 전에도 우크라이나에 계속 미사일 공격을 했다'며 '푸틴은 전투 패배로 절박한 상황이며 전황을 유리하게 바꾸려고 미사일 공포를 사용한다'고 반박했다.",
            "우크라이나 구조대원들이 10일(현지시간) 수도 키이우 시내에서 러시아군의 미사일 공격을 당한 현장을 조사하고 있다.",
            "이날 오전에 키이우 시내에 여러차례 폭발이 발생했다.",
            "푸틴 대통령은 이번 공격이 국방부의 요청에 따라 이뤄졌다고 주장했는데, 이것이 사실이라면 이는 새로운 합동군 총사령관 세르게이 수로비킨의 첫번째 결정이라고 가디언은 전했다.",
            "수로비킨 사령관과 함께 일했던 전 국방부 관계자는 가디언지에 '오늘 키이우에서 벌어진 일이 놀랍지 않았다.",
            "그는 매우 무자비하고 사람 목숨을 신경 쓰지 않는다'며 '그의 손이 우크라이나인의 피로 뒤덮일까 걱정된다'고 말했다.",
            "그러나 푸틴 대통령이 이날 공습으로 얻은 매파들의 호평은 오래가지 않을 수 있다.",
            "그는 이번과 같은 대규모 미사일 공격은 러시아 영토 공격 시 대응으로 남겨둘 것이라고 말했는데 강경파들은 전면전을 원하고 있다.",
            "정치 평론가인 세르게이 마르코프는 '러시아 여론은 대규모 공격과 우크라이나군이 사용할 가능성이 있는 인프라 완전파괴를 원한다'고 말했다.",
            "한편으론 비판 세력의 목소리는 그다지 의미가 없을 수도 있어 보인다.",
            "전쟁과 관련한 의사결정 과정은 여전히 불투명하다.",
            "콜레스니코프 선임 연구원은 '푸틴 대통령으로선 매파와 극보수파의 불만에 대응하는 것이 중요하겠지만 나는 그들의 영향력을 과장하진 않을 것'이라며 '푸틴 자신이 가장 매파적이고 극보수적인 인물'이라고 말했다.",
            "러시아에서 나오는 한 가지 이론은 푸틴 대통령이 악명 높은 새로운 군사령관을 임명함으로써 전쟁에서 국방부의 성과에 대한 분노를 줄이려고 한다는 것이다.",
            "수로비킨 사령관과 2020년까지 함께 일했던 전 공군 중위인 글렙 이리소프는 '수로비킨은 강경파들을 선호하고 와그너 용병회사와도 좋은 관계를 유지한다'며 '그러나 그가 매우 잔인한 동시에 유능한 사령관이지만 모든 문제를 풀 수는 없을 것'이라고 말했다."
        ]


# new_text = edit_sentences(STRTexT)
# # print('원본 split_sentences :', new_text)
# total_value = []
# for x in range(len(new_text)):
#     total_value.append(' '.join(edit_josa(new_text[x])))

# print(total_value)
TUNNEDText = ['러시아 우크라이나 키이우 등 공습 것 군 내부 비판 블라디미르 푸틴 러시아 대통령 자존심 때문 분석 나오',
              '영국 가디언지 10일 현지시간 푸틴 대통령 대규모 공습 국내 군 비판세력 러시아 침공 실패 있 사실 크림대교 폭발 후 상처 자존심 대하 절박 답변 풀이',
              '가디언 따르 미국 싱크탱크 카네기국제평화재단 선임 연구원 안드레이 콜레스니코프 지금 푸틴 하 것 사소 복수 이 개인 복수 있 고 말',
              '러시아 전쟁 전문가 군사 블로거 등 수개월간 우크라이나 상대로 전면전 벌이 촉구 키이우 등 거리 시신 있 끔찍 사진 나오 지금 만족',
              '최근 러시아군 수뇌부 비판 람잔 카디로프 체첸 공화국 수장 보 젤렌스 우크라이나 대통령 우리 러시아 아직 제대로 시작 안 경고 이 이제 전쟁 진행 100 만족 고 말',
              '푸틴 대통령 이날 러시아 안전보장 회의 주재 크림대교 폭발 배후 우크라이나 정보 당국 겨냥 이런 종류 범죄 대응 하 않 것 불가능 고 말',
              '그러나 드미트 쿨레바 우크라이나 외무장관 러시아 크림대교 사건 전 우크라이나 계속 미사일 공격 하 이 푸틴 전투 패배 절박 상황 전황 유리 바꾸 미사일 공포 사용 고 반박',
              '우크라이나 구조대원 10일 현지시간 수 키이우 시내 러시아군 미사일 공격 당하 현장 조사 있',
              '이날 오전 키이우 시내 여러차례 폭발 발생', 
              '푸틴 대통령 이번 공격 국방부 요청 따르 이루어지 주장 이것 사실 이 새롭 합동군 총사령관 세르 수로비킨 첫번 결정 가디언 전하', 
              '수로비킨 사령관 함께 일 전 국방부 관계자 가디언지 오늘 키이우 벌어지 일 놀랍 않', 
              '그 매우 무자비 사람 목숨 신경 쓰 않 이 그 손 우크라이나인 피로 뒤덮이 걱정 고 말', 
              '그러나 푸틴 대통령 이날 공습 얻 매파 호평 오래가 않 수 있', 
              '그 이번 같 대규모 미사일 공격 러시아 영토 공격 시 대응 남기 것 말 강경파 전면전 원하 있', 
              '정치 평론가 세르 마르코프 러시아 여론 대규모 공격 우크라이나군 사용 가능 있 인프라 완전파괴 원하 고 말', 
              '한편 비판 세력 목소리 그다지 의미 없 수 있 보이', 
              '전쟁 관련 의사결정 과정 여전히 불투명', 
              '콜레스니코프 선임 연구원 푸틴 대통령 매파 극보수파 불만 대응 것 중요 나 그 영향력 과장하진 않 것 이 푸틴 자신 가장 매파 극보수 인물 이 말', 
              '러시아 나오 한 가지 이론 푸틴 대통령 악명 높 새롭 군사령관 임명 전쟁 국방부 성과 대하 분노 줄이 하 것', 
              '수로비킨 사령관 2020년 함께 일 전 공군 중위 글렙 이리소프 수로비킨 강경파 선호 오 용병회사 좋 관계 유지 이 그러나 그 매우 잔인 동시 유능 사령관 모든 문제 풀 수 없 것 이 말']

빈도수 기반 명사 추출

In [3]:
komoran = Komoran()
# STRTexT에서 명사를 추출, 명사의 수를 count로 선언
noun = komoran.nouns(STRTexT)
count = Counter(noun)

# 명사 빈도 카운트
noun_list = count.most_common(20)
for v in noun_list:
    if len(v[0]) == 1:
        pass
    else:
        print("Komoran 기준 : " + str(v))

Komoran 기준 : ('러시아', 10)
Komoran 기준 : ('푸틴', 9)
Komoran 기준 : ('우크라이나', 8)
Komoran 기준 : ('대통령', 8)
Komoran 기준 : ('공격', 6)
Komoran 기준 : ('비판', 4)
Komoran 기준 : ('가디언', 4)
Komoran 기준 : ('전쟁', 4)
Komoran 기준 : ('미사일', 4)
Komoran 기준 : ('수로', 4)
Komoran 기준 : ('비킨', 4)
Komoran 기준 : ('이우', 3)
Komoran 기준 : ('공습', 3)


TF.IDF 

In [123]:
vectorizer = TfidfVectorizer()
vectorizer.fit(TUNNEDText)
matrix = vectorizer.transform(TUNNEDText)

vocabulary_word_id = defaultdict(int)
    
for idx, token in enumerate(vectorizer.get_feature_names()):
    vocabulary_word_id[token] = idx
    
# 특징 추출 결과: {"token": value}
result = defaultdict(str)
    
for token in vectorizer.get_feature_names():
    result[token] = matrix[0, vocabulary_word_id[token]]
    
# 내림차순 (중요도 high) 기준 정렬
result = sorted(result.items(), key = lambda item: item[1], reverse = True)
pprint(result[:20])

[('러시아', 0.34349799588831337),
 ('내부', 0.33043401009471474),
 ('때문', 0.33043401009471474),
 ('분석', 0.33043401009471474),
 ('블라디미르', 0.33043401009471474),
 ('자존심', 0.2904565526794969),
 ('공습', 0.2620920952438779),
 ('나오', 0.2620920952438779),
 ('비판', 0.2620920952438779),
 ('키이우', 0.22211463782865998),
 ('우크라이나', 0.20691591390621303),
 ('대통령', 0.1821371804134421),
 ('푸틴', 0.17174899794415668),
 ('100', 0.0),
 ('10일', 0.0),
 ('2020년', 0.0),
 ('가능', 0.0),
 ('가디언', 0.0),
 ('가디언지', 0.0),
 ('가장', 0.0)]


In [125]:
countList = []

tfidfv = TfidfVectorizer().fit(TUNNEDText)

for k,v in tfidfv.vocabulary_.items():
    textTuple = ((k, v))
    countList.append(textTuple)

countList.sort(reverse=True)
countList[:20]

[('회의', 197),
 ('호평', 196),
 ('현지시간', 195),
 ('현장', 194),
 ('합동군', 193),
 ('함께', 192),
 ('한편', 191),
 ('피로', 190),
 ('풀이', 189),
 ('푸틴', 188),
 ('폭발', 187),
 ('평론가', 186),
 ('패배', 185),
 ('키이우', 184),
 ('크림대교', 183),
 ('쿨레바', 182),
 ('콜레스니코프', 181),
 ('카디로프', 180),
 ('카네기국제평화재단', 179),
 ('침공', 178)]

Text Summarization

In [ ]:
# 해당 기능은 기계학습이 필요하기 때문에 후순위로 선정한다.

* Text Summarization With Attention

In [ ]:
# 해당 기능은 기계학습이 필요하기 때문에 후순위로 선정한다.

* TextRank Based on Sentence Embedding

In [ ]:
# 해당 기능은 기계학습이 필요하기 때문에 후순위로 선정한다.

RAKE

WordRank

* KR-WordRank

In [110]:
texts = TUNNEDText
# texts = LISTText
wordrank_extractor = KRWordRank(min_count=5, max_length=10)
keywords, rank, graph = wordrank_extractor.extract(texts, num_keywords=100)

def make_vocab_score(keywords, scaling=None):
    if scaling is None:
        scaling = lambda x:math.sqrt(x)
    return {word:scaling(rank) for word, rank in keywords.items()}

keywords = make_vocab_score(keywords)
print(keywords)

{'푸틴': 1.3732243555746098, '대통령': 1.3521268801052138, '러시아': 1.3020997627605881, '우크': 1.1216525136431523, '공격': 1.0979590292080694, '키이': 1.0581310608472447}


* sentence extraction

In [112]:
texts = LISTText
keywords, sents = summarize_with_sentences(texts, num_keywords=100, num_keysents=10)
print(sents)

["푸틴 대통령은 이날 러시아 안전보장이사회 회의를 주재하고 크림대교 폭발의 배후로 우크라이나 정보 당국을 겨냥하면서 '이런 종류의 범죄에 대응을 하지 않는 것은 불가능하다'고 말했다.", "정치 평론가인 세르게이 마르코프는 '러시아 여론은 대규모 공격과 우크라이나군이 사용할 가능성이 있는 인프라 완전파괴를 원한다'고 말했다.", '영국 가디언지는 10일(현지시간) 푸틴 대통령의 대규모 공습은 국내 군 비판세력, 러시아가 침공에서 실패하고 있다는 사실, 크림대교 폭발 후 상처받은 자존심에 대한 절박한 답변이라고 풀이했다.', "가디언에 따르면 미국 싱크탱크 카네기국제평화재단의 선임 연구원 안드레이 콜레스니코프는 '지금 푸틴이 하는 것은 사소한 복수'라며 '개인적 복수도 있다'고 말했다.", '우크라이나 구조대원들이 10일(현지시간) 수도 키이우 시내에서 러시아군의 미사일 공격을 당한 현장을 조사하고 있다.', '푸틴 대통령은 이번 공격이 국방부의 요청에 따라 이뤄졌다고 주장했는데, 이것이 사실이라면 이는 새로운 합동군 총사령관 세르게이 수로비킨의 첫번째 결정이라고 가디언은 전했다.', "수로비킨 사령관과 2020년까지 함께 일했던 전 공군 중위인 글렙 이리소프는 '수로비킨은 강경파들을 선호하고 와그너 용병회사와도 좋은 관계를 유지한다'며 '그러나 그가 매우 잔인한 동시에 유능한 사령관이지만 모든 문제를 풀 수는 없을 것'이라고 말했다.", '이날 오전에 키이우 시내에 여러차례 폭발이 발생했다.', '한편으론 비판 세력의 목소리는 그다지 의미가 없을 수도 있어 보인다.', '전쟁과 관련한 의사결정 과정은 여전히 불투명하다.']


TextRank 

TopicRank 

TopicalPageRank 

PositionRank 

MultipartiteRank  

ExpandRank 

KEA 

KeyBERT 